# DeepFake using first-order-motion-model

**Clone repository**

In [12]:
# !git clone https://github.com/AliaksandrSiarohin/first-order-model.git

In [1]:
cd first-order-model

/Users/devasenan/Documents/projects/hologram_with_speech_synthesis/first-order-motion-model/first-order-model


In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from math import ceil
print("done import")

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
done import


**Utils**

In [3]:
# root_path = "/content/gdrive/MyDrive/ColabNotebooks/ML/first-order-motion-model/data"
root_path = "../../data"

def img_vid_resize(source, cap, size_x, size_y):
    source_image = cv2.resize(source, (size_x, size_y))
    driving_video = "vid_in/driving.avi"
    fps = cap.get(cv2.CAP_PROP_FPS)
    # print(fps)

    fourcc = cv2.VideoWriter_fourcc(*"XVID")
    writer = cv2.VideoWriter(
        f"{root_path}/{driving_video}", fourcc, fps, (size_x, size_y))

    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, (size_x, size_y))
            # cv2.imshow("img", source_image)
            # cv2.imshow("vid", frame)
            writer.write(frame)

            if cv2.waitKey(ceil(fps)) == ord("q"):
                break
        else:
            break
    
    cv2.destroyAllWindows()
    cap.release()
    writer.release()
    return (source_image, driving_video)


def display_plot(source, driving_video, generated=None):
    titles = ["source_image", "driving_video"]

    driving = cv2.VideoCapture(f"{root_path}/{driving_video}")
    video_length = driving.get(cv2.CAP_PROP_FRAME_COUNT)
    source = cv2.cvtColor(source, cv2.COLOR_BGR2RGB)

    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))
    ims = []
    for i in range(int(video_length)):
        cols = [source]
        cols.append(cv2.cvtColor(driving.read()[1], cv2.COLOR_BGR2RGB))
        if generated is not None:
            cols.append(cv2.cvtColor(generated[1], cv2.COLOR_BGR2RGB))
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)

    plt.close()
    return ani

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani


**Displaying source image and driving video**

In [7]:
from IPython.display import HTML

image = cv2.imread(root_path + "/img_in/Trump.png")
cap = cv2.VideoCapture(root_path + "/vid_in/bill-gates.mp4")
x, y = 256, 256                         # resizing the image and video
cv2.imshow("test", image)
source_image, driving_video = img_vid_resize(image, cap, x, y)
print("done resize")

HTML(display_plot(source_image, driving_video).to_html5_video())

In [6]:
%conda install -c conda-forge ffmpeg

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.3.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /Users/devasenan/opt/anaconda3/envs/tensor

  added / updated specs:
    - ffmpeg


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ffmpeg-4.2.2               |       h97e5cf8_0        22.9 MB
    gettext-0.21.1             |       h8a4c099_0         4.0 MB  conda-forge
    lame-3.100                 |    hb7f2c08_1003         530 KB  conda-forge
    libiconv-1.17              |       hac89ed1_0         1.3 MB  conda-forge
    libidn2-2.3.4              |       hb7f2c08_0         170 KB  conda-forge
    libopus-1.3.1              |       hc929b4f_1         273 KB  conda-forge
    libtasn1-4.19.0            |       hb7f2c08_0         116 K

**Create a model and load checkpoints**

In [ ]:
from IPython.display import HTML
from demo import load_checkpoints

generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='/content/gdrive/MyDrive/ColabNotebooks/ML/first-order-motion-model/data/vox-cpk.pth.tar')

**Perform image animation**

In [ ]:
from demo import make_animation
from IPython.display import HTML

cap = cv2.VideoCapture(f"{root_path}/{driving_video}")
driving = []
while cap.isOpened():
  ret, frame = cap.read()
  if ret:
    driving.append(frame)
  else:
    break

predictions = make_animation(source_image, driving, generator, kp_detector, relative=True)
# predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=False, adapt_movement_scale=True)

# save resulting video
## imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)
#video can be downloaded from /content folder


HTML(display(source_image, driving, predictions).to_html5_video())